<a href="https://colab.research.google.com/github/WuilsonEstacio/Procesamiento-de-lenguaje-natural/blob/main/procesamiento_de_lenguaje_natural_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Un NB clasificador  probabilistico es aque que dado un documeto cualquiera le asiga una probabilidad para cada una de las categorias en su interior

$c= arg max_c * \dfrac{P(d/c)P(c}{P(d)}= arg max_c *{P(d/c)P(c)}$

$P(d/c)= P(f1,f2,f3... fn | c)=P(f1/c)P(f2/c)...P(fn/c) $ es ta es la probabilidad de que dada una categoria encontraramos un documento, fi son features o atributos

$c= arg max_c *P(c)\prod_i^n P(fi/c) $
aplicaremos un log para evitar que el numero se valla a cero a infinito por lo tanto

$c= arg max_c*Log(P(c)\prod_i^n P(fi/c))$

$c= arg max_c*Log(P(c))+\sum Log(P(fi/c)) $

notya: underflow: Sobrepasar la precisión de máquina con un espacio logaritmico

In [ ]:
import math
import os

In [ ]:
!git clone https://github.com/pachocamacho1990/datasets

In [ ]:
! unzip datasets/email/plaintext/corpus1.zip

In [ ]:
os.listdir('corpus1/spam')

In [ ]:
data = []
clases = [] # esta lista nos dice si la etiqueta es spam o no lo es 

# lectura de spam datal, pare ello usamos la libreria  os. para recorres una lista de archivos 
for file in os.listdir('corpus1/spam'): # usamos el comando listdir para listar los archivos que encuentre en el directorio  corpus1/spam
  # ahora abrimos cada interaccion del for. encoding='latin-1'  por si se encuentran caracteres especiales
  with open('corpus1/spam/'+file, encoding='latin-1') as f:
    data.append(f.read()) # a la lista data le agrego el texto del archivo
    clases.append('spam') # le agrego la clase de ese archivo

# realizaremos lectura de ham data
for file in os.listdir('corpus1/ham'):
  with open('corpus1/ham/'+file, encoding='latin-1') as f:
    data.append(f.read())
    clases.append('ham')
len(data) # nos muestra la longitud de data

# **Construcción de modelo Naive Bayes**

# Tokenizador de Spacy
Documentación: <a href= https://spacy.io/api/tokenizer >Spacy</a> Esto permite implementar flujos de procesamiento, para llevar modelos de procesamiento de lenguaje a produccion

¿Cómo funciona el tokenizador? <a href=https://spacy.io/usage/linguistic-features#how-tokenizer-works>tokenizador_doc_spacy</a> 


In [ ]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

nlp = English()
tokenizer = Tokenizer(nlp.vocab) # vocab es Un contenedor de almacenamiento para tipos léxicos..

In [ ]:
print([t.text for t in tokenizer(data[0])])

Clase principal para el algoritmo
Recuerda que la clase más probable viene dada por (en espacio de cómputo logarítmico):

$c^=argmax(c)logP(c)+∑i=1nlogP(fi|c)$ 

Donde, para evitar casos atípicos, usaremos el suavizado de Laplace así:

$P(fi|c)=\dfrac{C(fi,c)+1}{ C(c)+|V|}$

siendo $ |V|$  la longitud del vocabulario de nuestro conjunto de entrenamiento.

In [ ]:
# construccion del algoritmo
import numpy as np

class NaiveBayesClassifier(): # construccion de la clase 
  nlp = English() # atributos de la clase
  tokenizer = Tokenizer(nlp.vocab) # atributos de la clase
  
  # retorna el documento tokenizado en una lista
  def tokenize(self, doc): 
    return  [t.text.lower() for t in tokenizer(doc)]

# contador de palabras
  def word_counts(self, words):
    wordCount = {} # diccionario basio
    for w in words: # recorro tofdas las palabras
      if w in wordCount.keys(): # verificamos si la palabra esta en el diccionario
        wordCount[w] += 1 # si lo anterior sucede le sumamos una llave adecional
      else:
        wordCount[w] = 1 # de lo contrario creamos la llave
    return wordCount # debemos retornar un conteo

# ajustamos el modelo para poder entrenarlo y calcular todas sus probabilidades
  def fit(self, data, clases):
    n = len(data) # definimos la longitud de los datos de entrenamiento
    self.unique_clases = set(clases) # identificamos clases unicas
    self.vocab = set() # creamos atributo para asignar el vocabulario
    self.classCount = {} # C(c) Creamos una clase para contar el numero de bese que aparece cada categoria posible en el corpus
    self.log_classPriorProb = {} #P(c) calculamos el log de la probabilidad de las clases
    self.wordConditionalCounts = {} #C(w|c) probabilidad de las clases condicionales
   
    #Conteos de clases
    for c in clases: # repetimos lo de arriba
      if c in self.classCount.keys():
        self.classCount[c] += 1
      else:
        self.classCount[c] = 1

    # Calculo de P(c)
    for c in self.classCount.keys():
      self.log_classPriorProb[c] = math.log(self.classCount[c]/n)
      self.wordConditionalCounts[c] = {}

    # calculo de C(w|c)
    for text, c in zip(data, clases):
       # por cada interacion del for se hace un conteo del texto tokenizado
      counts = self.word_counts(self.tokenize(text))
      for word, count in counts.items(): # este for es para contar el numero de veces que aparece cada palabra
        if word not in self.vocab: # verificamos que la palabra no este en el diccionario  self.vocab.add(word)
          self.vocab.add(word) # y si no esta la agregamos
        if word not in self.wordConditionalCounts[c]: # si la palabra no esta en los conteos condicionales
          self.wordConditionalCounts[c][word] = 0.0 # si word no esta alli pues agregamos el elemento de la llave
        self.wordConditionalCounts[c][word] += count
        
# para poder predecir las clases
  def predict(self, data):
    results = [] # creamos lista vacia para almacenar
    for text in data:
      words = set(self.tokenize(text)) #
      scoreProb = {} # definimos diccionario donde cada clase tiene su probabilidad
      for word in words: 
        if word not in self.vocab: continue #esti nos dice que si la palabra no esta en el diccionario entonces la ignoramos
       # si la palabra si esta en el diccionario entonces la suabizamos
        #suavizado Laplaciano para P(w|c)
        for c in self.unique_clases:
          log_wordClassProb = math.log(
              (self.wordConditionalCounts[c].get(word, 0.0)+1)/(self.classCount[c]+len(self.vocab)))
          scoreProb[c] = scoreProb.get(c, self.log_classPriorProb[c]) + log_wordClassProb
      arg_maxprob = np.argmax(np.array(list(scoreProb.values())))
      '''en la sig linea agregamos la maxima probabilidad correspondiente a ese argumento
      esto se hace con scoreProb.keys() donde cogemos la lista y de esa lista cogemos la posiscion 
      donde al agumento me dio la maxima probabilidad'''
      results.append(list(scoreProb.keys())[arg_maxprob])
    return results

Utilidades de Scikit Learn
train_test_split: <a href = https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html>AQUI</a>

accuracy_score: <a href =https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html>AQUI</a>

precision_score:<a href = https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html>AQUI</a>

recall_score: <a href = https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html>AQUI</a>

**Verificaremos si el algoritmo funciona correctamente**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
data_train, data_test, clases_train, clases_test = train_test_split(data, clases, test_size=0.10, random_state=42)

In [ ]:
classifier = NaiveBayesClassifier()
classifier.fit(data_train, clases_train) # ajustamos los datos y clases de entrenamiento

In [ ]:
clases_predict = classifier.predict(data_test)

In [ ]:
# Acurracy
accuracy_score(clases_test, clases_predict) # accuracy_score lo usamos para comparar las clases del test, encomparacion con las clases predichas

In [ ]:
# precision
# esto indica que cuanto de los selecionados como  ham y spam son verdaderamente spam
# precicion (ham, spam)
precision_score(clases_test, clases_predict, average=None, zero_division=1)

In [ ]:
# recall
# esto nos indica cuantos datos de ham y spam logramosidentificar bien
# recall(ham,spam)
recall_score(clases_test, clases_predict, average=None, zero_division=1)

$Accuracy(y,y')=\dfrac{1}{n} \sum_{i=1}^n \Upsilon_{y_i,y_i´}$

el acurracy es la proporcion entre el numero de prediccions que fueron correctas y el numero total de predecciones realizadas si 
esta es =1 si son iguales

Acurracy = TP+ TN


$Precicion = \dfrac{TP}{TP+FP}$ esto indica que cuanto de los selecionados como x son verdaderamente x


$Recall=\dfrac{TP}{TP+FN}$ esto nos indica cuantos datos x logramosidentificar bien

# **Test** 

1.
Si tenemos un dataset etiquetado donde la categoría adjetivo (ADJ) aparece un total de 500 veces entre todos los tokens, y de esas veces solamente la palabra "noble" le corresponde 200 veces, entonces podemos decir que:

La probabilidad de emisión P(noble|ADJ) = 40%

2.
El proceso mediante el cual un Modelo Markoviano Latente determina la secuencia de etiquetas más probable para una secuencia de palabras es:

Usando el algoritmo de Viterbi para obtener la categoría más probable, palabra por palabra.

3.
Dada una cadena de texto text en español, el procedimiento para asignar las etiquetas gramaticales con Stanza es a partir de un objeto nlp(text), donde:

nlp = stanza.Pipeline('es', processors='tokenize,pos')

4.
La ingeniería de atributos se usa para:

Construir atributos particulares de palabras y textos que permitan dar un input más apropiado a un modelo de clasificación.

5.
El problema de clasificación de texto pertenece a la categoría de Machine Learning supervisado porque:

Durante el entrenamiento, el modelo tiene conocimiento de las etiquetas correctas que debería predecir.

6.
En un modelo de clasificación por categorías gramaticales, el algoritmo de Viterbi se usa para:

El proceso de decodificación: encontrar la secuencia de etiquetas más probable.

7.
En un Modelo Markoviano Latente se necesitan los siguientes ingredientes:

Matrices de transición, emisión y distribución inicial de estados.

8.
En un problema de clasificación de emails entre SPAM y HAM, la métrica de recall tiene la siguiente interpretación:

De todos los correos que realmente son SPAM, la fracción que el modelo logró identificar.

9.
Para entrenar un clasificador de Naive Bayes en NLTK, se escribe en Python:

nltk.NaiveBayesClassifier.train(data)

10.
Si tienes un modelo de clasificación binaria que luego de entrenarlo, obtienes que el número de verdaderos positivos es 200 y el número de falsos positivos es 120, entonces la métrica de precisión de dicho modelo tiene un valor de:

200/320

11.
Un algoritmo general de clasificación de texto:

Es un algoritmo de Machine Learning supervisado.

12.
El tokenizador por defecto en NLTK para el idioma inglés es:

punkt

13.
En una cadena de Markov se necesitan los siguientes elementos:

Matriz de transiciones y distribución inicial de estados.

14.
Entrenar un Modelo Markoviano Latente significa:

Calcular las matrices de probabilidad de transición y emisión con un corpus de textos.

15.
Una de las siguientes no es una categoría de ambigüedades del lenguaje:

Vectorial

16.
El suavizado de Laplace se usa en un algoritmo de clasificación con el objetivo de:

Evitar probabilidades nulas y denominadores iguales a cero.

17.
El clasificador de Naive Bayes es:

Un clasificador probabilístico que hace uso de la regla de Bayes.

18.
En la frase: "mi hermano es muy noble", la palabra noble hace referencia a:

Un adjetivo

19.
Con Naive Bayes preferimos hacer cálculos en espacio logarítmico para:

Evitar productos de números demasiado pequeños para la precisión de máquina.

20.
En un modelo MEMM:

El proceso de decodificación es similar al de un HMM, y por lo tanto se puede usar un tipo de algoritmo de Viterbi.

21.
El accuracy de entrenamiento de un modelo se calcula como:

(número de veces que el modelo predice la categoría correcta) / (total de datos usados para entrenamiento)

22.
Si tenemos una cadena de Markov para describir las probabilidades de transición en cuanto al clima de un dia para otro, y observamos la siguiente secuencia de estados día tras día: (frío, frío, caliente, frío, tibio, caliente, tibio, frío), entonces la probabilidad de transición P(caliente|frío) es:

50%

23.

En un Modelo Markoviano Latente, el problema de calcular la secuencia de etiquetas más probable se expresa con la siguiente expresión matemática:

$${\arg \max}_{(t^n)}\prod_i P(w_i \vert t_i)P(t_i \vert t_{i-1})$$

24.
Para un modelo de clasificación de palabras con Naive Bayes en NLTK, debemos entrenar el algoritmo usando:

nltk.NaiveBayesClassifier.train(train_set) donde usamos una funcion que extrae atributos llamada atributos() y:

train_set = [(atributos(palabra), categoría de la palabra), ...]

25.
Dada una cadena de texto text en inglés, el procedimiento para asignar las etiquetas gramaticales con NLTK es:

nltk.pos_tag(word_tokenize(text))